In [2]:
import numpy as np
import pandas as pd

### Poplabels file

In [3]:
sample = pd.read_table(snakemake.input['sample']).iloc[1:]

In [4]:
poplabels = pd.DataFrame({
    'sample': sample.ID_1,
    'population': 'Turkana',
    'group': 'Turkana',
    'sex': 'NA'
})

In [5]:
poplabels.to_csv(snakemake.output['poplabels'], sep=' ', index=False)

### Repolarize

So that the derived allele is the one at the highest frequency. We do this because we know our sweep to be at ~80% frequency, and we want to be able to narrow our analyses to that.

In [6]:
with open(snakemake.output["repolarized"], 'w') as out:
    with open(snakemake.input["haps"]) as haps:
        for line in haps:
            cols = line.strip().split()
            chrom = cols[0]
            snp_id = cols[1]
            pos = int(cols[2])
            allele_0 = cols[3]
            allele_1 = cols[4]
            haplotypes = np.array(cols[5:], dtype=np.uint16)
            freq = sum(haplotypes)/len(haplotypes)
            if freq < 0.5:
                haplotypes = 1 - haplotypes
            out.write(f"{chrom} {snp_id} {pos} {allele_0} {allele_1} {' '.join(str(h) for h in haplotypes)}" + '\n')